In [1]:
import sys
sys.path.append('./KPConv-PyTorch/')

In [2]:
from utils.config import Config
from datasets.CellData import CellDataset, CellDataCollate, CellDataCustomBatch, CellDataSampler
from models.architectures import KPFCNN
from torch.utils.data import DataLoader
from utils.trainer import ModelTrainer
import numpy as np
import torch

In [3]:
class CellDataConfig(Config):
    """
    Override the parameters you want to modify for this dataset
    """

    ####################
    # Dataset parameters
    ####################

    # Dataset name
    dataset = 'Fluo-C3DH-A549'

    # Number of classes in the dataset (This value is overwritten by dataset class when Initializating dataset).
    num_classes = None

    # Type of task performed on this dataset (also overwritten)
    dataset_task = 'cloud_segmentation'

    # Number of CPU threads for the input pipeline
    input_threads = 0

    #########################
    # Architecture definition
    #########################

    # Define layers
    architecture = ['simple',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary']

    ###################
    # KPConv parameters
    ###################

    # Radius of the input sphere
    in_radius = 3.0

    # Number of kernel points
    num_kernel_points = 15

    # Size of the first subsampling grid in meter
    first_subsampling_dl = 0.3

    # Radius of convolution in "number grid cell". (2.5 is the standard value)
    conv_radius = 2.75

    # Radius of deformable convolution in "number grid cell". Larger so that deformed kernel can spread out
    deform_radius = 6.0

    # Radius of the area of influence of each kernel point in "number grid cell". (1.0 is the standard value)
    KP_extent = 1.5

    # Behavior of convolutions in ('constant', 'linear', 'gaussian')
    KP_influence = 'linear'

    # Aggregation function of KPConv in ('closest', 'sum')
    aggregation_mode = 'sum'

    # Choice of input features
    first_features_dim = 128
    in_features_dim = 1

    # Can the network learn modulations
    modulated = False

    # Batch normalization parameters
    use_batch_norm = True
    batch_norm_momentum = 0.02

    # Deformable offset loss
    # 'point2point' fitting geometry by penalizing distance from deform point to input points
    # 'point2plane' fitting geometry by penalizing distance from deform point to input point triplet (not implemented)
    deform_fitting_mode = 'point2point'
    deform_fitting_power = 1.0              # Multiplier for the fitting/repulsive loss
    deform_lr_factor = 0.1                  # Multiplier for learning rate applied to the deformations
    repulse_extent = 1.2                    # Distance of repulsion for deformed kernel points

    #####################
    # Training parameters
    #####################

    # Maximal number of epochs
    max_epoch = 500

    # Learning rate management
    learning_rate = 1e-6
    momentum = 0.98
    lr_decays = {i: 0.1 ** (1 / 150) for i in range(1, max_epoch)}
    grad_clip_norm = 100.0

    # Number of batch
    batch_num = 10

    # Number of steps per epochs
    epoch_steps = 200

    # Number of validation examples per epoch
    validation_size = 50

    # Number of epoch between each checkpoint
    checkpoint_gap = 20

    # Augmentations
    augment_scale_anisotropic = True
    augment_symmetries = [True, False, False]
    augment_rotation = 'vertical'
    augment_scale_min = 0.8
    augment_scale_max = 1.2
    augment_noise = 0.001
    augment_color = 0.8

    # The way we balance segmentation loss
    #   > 'none': Each point in the whole batch has the same contribution.
    #   > 'class': Each class has the same contribution (points are weighted according to class balance)
    #   > 'batch': Each cloud in the batch has the same contribution (points are weighted according cloud sizes)
    segloss_balance = 'class'
    class_w = [0.02, 0.98]

    # Do we need to save convergence
    saving = True
    saving_path = None

In [4]:
config = CellDataConfig()
train_dataset = CellDataset(config, set = 'training')
validation_dataset = CellDataset(config, set = 'validation')

training_sampler = CellDataSampler(train_dataset)
validation_sampler = CellDataSampler(validation_dataset)


Preparing KDTree for cloud t000, subsampled at 0.300
14.7 MB loaded in 2.9s

Preparing KDTree for cloud t001_2, subsampled at 0.300
14.7 MB loaded in 3.0s

Preparing KDTree for cloud t002, subsampled at 0.300
14.7 MB loaded in 2.9s

Preparing KDTree for cloud t006, subsampled at 0.300
14.7 MB loaded in 2.9s

Preparing KDTree for cloud t008, subsampled at 0.300
14.7 MB loaded in 3.1s

Preparing KDTree for cloud t008_2, subsampled at 0.300
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t011_2, subsampled at 0.300
14.7 MB loaded in 3.1s

Preparing KDTree for cloud t013, subsampled at 0.300
14.7 MB loaded in 3.5s

Preparing KDTree for cloud t013_2, subsampled at 0.300
14.7 MB loaded in 4.0s

Preparing KDTree for cloud t014, subsampled at 0.300
14.7 MB loaded in 3.9s

Preparing KDTree for cloud t015, subsampled at 0.300
14.7 MB loaded in 3.4s

Preparing KDTree for cloud t015_2, subsampled at 0.300
14.7 MB loaded in 3.2s

Preparing KDTree for cloud t019, subsampled at 0.300
14.7 MB load

In [5]:
training_loader = DataLoader(train_dataset,
                            collate_fn=CellDataCollate,
                            sampler=training_sampler,)

validation_loader = DataLoader(validation_dataset,
                              collate_fn=CellDataCollate,
                              sampler=training_sampler,)

training_sampler.calibration(training_loader)
validation_sampler.calibration(validation_loader)

net = KPFCNN(config, train_dataset.label_values, train_dataset.ignored_labels)
trainer = ModelTrainer(net, config)


Starting Calibration (use verbose=True for more details)
Calibration done in 10.6s


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s



In [ ]:
trainer.train(net, training_loader, validation_loader, config)

e000-i0000 => L=16.397 acc= 41% / t(ms):  62.3 4534.6 319.6)
e000-i0002 => L=16.146 acc= 43% / t(ms):  50.4 238.9 271.3)
e000-i0004 => L=16.262 acc= 50% / t(ms):  48.8 236.7 274.7)
e000-i0006 => L=15.996 acc= 45% / t(ms):  48.6 232.6 281.2)
e000-i0008 => L=15.964 acc= 45% / t(ms):  47.8 228.7 286.5)
e000-i0010 => L=15.951 acc= 50% / t(ms):  47.8 225.4 285.8)
e000-i0012 => L=15.948 acc= 46% / t(ms):  46.3 224.0 288.5)
e000-i0014 => L=16.025 acc= 45% / t(ms):  45.9 226.2 281.6)
e000-i0016 => L=16.237 acc= 50% / t(ms):  45.3 228.5 283.2)
e000-i0018 => L=16.341 acc= 46% / t(ms):  44.1 225.3 282.8)
e000-i0020 => L=16.313 acc= 48% / t(ms):  44.3 226.5 282.7)
e000-i0022 => L=15.907 acc= 54% / t(ms):  44.8 225.6 283.9)
e000-i0024 => L=15.989 acc= 50% / t(ms):  45.1 226.4 283.2)
e000-i0026 => L=15.815 acc= 43% / t(ms):  44.8 227.0 281.8)
e000-i0028 => L=15.902 acc= 42% / t(ms):  44.8 226.4 280.0)
e000-i0030 => L=15.860 acc= 40% / t(ms):  44.7 230.4 283.5)
e000-i0032 => L=15.742 acc= 46% / t(ms)

e001-i0020 => L=13.246 acc= 44% / t(ms):  48.2 271.1 288.1)
e001-i0022 => L=13.018 acc= 46% / t(ms):  48.0 267.0 287.6)
e001-i0024 => L=12.896 acc= 39% / t(ms):  47.2 264.4 282.5)
e001-i0026 => L=13.613 acc= 44% / t(ms):  47.3 266.3 285.9)
e001-i0028 => L=12.779 acc= 39% / t(ms):  47.2 265.7 287.5)
e001-i0030 => L=13.199 acc= 40% / t(ms):  47.3 270.6 296.8)
e001-i0032 => L=13.395 acc= 43% / t(ms):  49.2 278.8 310.4)
e001-i0034 => L=13.008 acc= 42% / t(ms):  51.2 289.7 324.7)
e001-i0036 => L=13.519 acc= 48% / t(ms):  54.9 301.4 334.7)
e001-i0038 => L=13.490 acc= 48% / t(ms):  55.4 303.1 342.7)
e001-i0040 => L=12.767 acc= 37% / t(ms):  55.5 307.4 343.5)
e001-i0042 => L=12.949 acc= 52% / t(ms):  56.3 310.9 344.1)
e001-i0044 => L=12.763 acc= 42% / t(ms):  57.3 311.1 341.6)
e001-i0046 => L=12.855 acc= 39% / t(ms):  56.9 314.0 346.4)
e001-i0048 => L=12.967 acc= 40% / t(ms):  57.3 314.2 344.0)
e001-i0050 => L=12.853 acc= 40% / t(ms):  57.9 312.9 349.3)
e001-i0052 => L=12.802 acc= 41% / t(ms):

e002-i0042 => L=12.641 acc= 43% / t(ms):  44.0 252.9 297.9)
e002-i0044 => L=12.294 acc= 42% / t(ms):  43.5 249.9 294.8)
e002-i0046 => L=12.827 acc= 42% / t(ms):  42.1 247.0 286.0)
e002-i0048 => L=12.344 acc= 44% / t(ms):  41.1 242.3 289.1)
e002-i0050 => L=12.787 acc= 42% / t(ms):  40.6 238.1 292.7)
e002-i0052 => L=12.791 acc= 44% / t(ms):  40.7 234.2 290.0)
e002-i0054 => L=12.080 acc= 41% / t(ms):  41.2 232.5 290.4)
e002-i0056 => L=11.933 acc= 38% / t(ms):  41.2 237.9 291.4)
e002-i0058 => L=12.421 acc= 39% / t(ms):  42.8 246.8 296.6)
e002-i0060 => L=11.907 acc= 38% / t(ms):  44.0 256.9 302.9)
e002-i0062 => L=12.135 acc= 38% / t(ms):  44.6 261.9 310.3)
e002-i0064 => L=12.213 acc= 38% / t(ms):  45.7 266.6 312.7)
e002-i0066 => L=12.351 acc= 43% / t(ms):  46.6 270.5 313.1)
e002-i0068 => L=12.981 acc= 41% / t(ms):  47.1 269.9 312.3)
e002-i0070 => L=12.091 acc= 36% / t(ms):  47.3 273.0 316.4)
e002-i0072 => L=13.015 acc= 46% / t(ms):  48.0 275.6 313.8)
e002-i0074 => L=12.269 acc= 42% / t(ms):

e003-i0066 => L=12.832 acc= 41% / t(ms):  59.4 293.4 341.2)
e003-i0068 => L=11.596 acc= 42% / t(ms):  58.1 292.6 334.4)
e003-i0070 => L=12.588 acc= 42% / t(ms):  56.5 292.5 333.6)
e003-i0072 => L=11.895 acc= 42% / t(ms):  55.4 294.1 327.4)
e003-i0074 => L=11.768 acc= 39% / t(ms):  54.3 292.2 329.9)
e003-i0076 => L=11.683 acc= 42% / t(ms):  53.1 288.9 333.3)
e003-i0078 => L=11.530 acc= 40% / t(ms):  53.3 288.4 330.1)
e003-i0080 => L=11.665 acc= 41% / t(ms):  51.8 284.2 331.9)
e003-i0082 => L=12.711 acc= 46% / t(ms):  50.4 286.9 335.3)
e003-i0084 => L=11.463 acc= 53% / t(ms):  48.6 276.6 324.3)
e003-i0086 => L=11.533 acc= 38% / t(ms):  47.1 263.4 317.1)
e003-i0088 => L=12.781 acc= 44% / t(ms):  45.2 255.0 316.5)
e003-i0090 => L=12.827 acc= 41% / t(ms):  43.5 247.5 308.1)
e003-i0092 => L=11.990 acc= 39% / t(ms):  42.8 242.7 305.1)
e003-i0094 => L=11.657 acc= 44% / t(ms):  42.9 247.4 304.8)
e003-i0096 => L=12.808 acc= 47% / t(ms):  42.3 243.7 306.6)
e003-i0098 => L=11.638 acc= 53% / t(ms):

e004-i0092 => L=12.324 acc= 57% / t(ms):  42.3 243.8 285.7)
e004-i0094 => L=11.659 acc= 47% / t(ms):  41.8 239.1 279.4)
e004-i0096 => L=12.067 acc= 45% / t(ms):  41.3 235.9 280.8)
e004-i0098 => L=11.325 acc= 48% / t(ms):  40.6 237.6 285.1)
e004-i0100 => L=11.490 acc= 51% / t(ms):  41.5 240.9 286.8)
e004-i0102 => L=11.261 acc= 53% / t(ms):  42.1 249.9 299.2)
e004-i0104 => L=12.299 acc= 49% / t(ms):  42.8 260.6 305.7)
e004-i0106 => L=12.527 acc= 51% / t(ms):  44.0 266.3 310.8)
e004-i0108 => L=11.509 acc= 49% / t(ms):  45.2 269.5 314.2)
e004-i0110 => L=11.619 acc= 50% / t(ms):  47.0 272.4 316.6)
e004-i0112 => L=11.755 acc= 48% / t(ms):  47.1 276.9 322.0)
e004-i0114 => L=12.406 acc= 53% / t(ms):  46.7 278.2 321.1)
e004-i0116 => L=12.216 acc= 50% / t(ms):  47.2 282.8 320.8)
e004-i0118 => L=11.410 acc= 53% / t(ms):  48.0 285.8 327.1)
e004-i0120 => L=11.741 acc= 43% / t(ms):  49.0 298.0 328.2)
e004-i0122 => L=12.248 acc= 50% / t(ms):  52.1 298.4 328.5)
e004-i0124 => L=11.841 acc= 40% / t(ms):

...

Validation : 372.0% (timings : 60.91 120.48)
Validation : 384.0% (timings : 60.03 119.66)
Validation : 394.0% (timings : 61.92 125.77)
Confusion Matrix
****************
[[4.9884244e+05 1.3052566e+04]
 [1.1934174e+04 4.5882568e+02]]
[0.95229983 0.01803165]
Fluo-C3DH-A549 mean IoU = 48.5%
e072-i0000 => L=9.012 acc= 93% / t(ms): 33600.1 311.2 305.2)
e072-i0002 => L=8.782 acc= 93% / t(ms):  47.5 327.7 356.2)
e072-i0004 => L=8.583 acc= 95% / t(ms):  50.6 323.6 355.0)
e072-i0006 => L=8.678 acc= 97% / t(ms):  50.1 317.6 355.9)
e072-i0008 => L=8.993 acc= 94% / t(ms):  49.4 309.5 355.4)
e072-i0010 => L=8.757 acc= 96% / t(ms):  49.2 294.9 339.3)
e072-i0012 => L=9.002 acc= 95% / t(ms):  47.3 279.2 322.7)
e072-i0014 => L=8.985 acc= 94% / t(ms):  46.4 274.0 313.5)
e072-i0016 => L=8.946 acc= 94% / t(ms):  44.9 264.1 302.1)
e072-i0018 => L=8.562 acc= 97% / t(ms):  43.3 258.4 294.6)
e072-i0020 => L=8.747 acc= 88% / t(ms):  42.1 250.1 291.5)
e072-i0022 => L=8.875 acc= 95% / t(ms):  42.1 244.2 289.7)
e0

e073-i0016 => L=8.890 acc= 89% / t(ms):  39.5 225.6 269.3)
e073-i0018 => L=9.236 acc= 88% / t(ms):  39.2 225.2 276.1)
e073-i0020 => L=9.556 acc= 88% / t(ms):  38.9 226.5 276.5)
e073-i0022 => L=9.512 acc= 90% / t(ms):  40.3 227.0 278.9)
e073-i0024 => L=8.820 acc= 95% / t(ms):  40.2 225.9 281.2)
e073-i0026 => L=8.862 acc= 94% / t(ms):  39.8 229.9 279.7)
e073-i0028 => L=9.254 acc= 94% / t(ms):  41.1 236.1 293.2)
e073-i0030 => L=9.608 acc= 81% / t(ms):  42.4 247.3 304.6)
e073-i0032 => L=9.076 acc= 93% / t(ms):  44.8 257.6 307.9)
e073-i0034 => L=8.932 acc= 86% / t(ms):  45.7 261.7 316.6)
e073-i0036 => L=9.604 acc= 92% / t(ms):  45.9 262.9 318.7)
e073-i0038 => L=8.668 acc= 92% / t(ms):  48.3 269.1 321.6)
e073-i0040 => L=9.006 acc= 91% / t(ms):  47.9 281.2 329.5)
e073-i0042 => L=9.174 acc= 93% / t(ms):  47.3 282.2 342.1)
e073-i0044 => L=8.583 acc= 96% / t(ms):  48.0 281.4 342.5)
e073-i0046 => L=9.254 acc= 93% / t(ms):  49.2 284.9 337.4)
e073-i0048 => L=9.131 acc= 91% / t(ms):  49.5 286.2 337.

e074-i0046 => L=9.067 acc= 87% / t(ms):  49.2 290.5 330.3)
e074-i0048 => L=9.720 acc= 93% / t(ms):  48.6 289.1 326.9)
e074-i0050 => L=8.974 acc= 97% / t(ms):  49.0 289.6 326.4)
e074-i0052 => L=8.859 acc= 96% / t(ms):  48.8 292.6 329.7)
e074-i0054 => L=9.069 acc= 92% / t(ms):  49.0 294.3 331.8)
e074-i0056 => L=8.772 acc= 95% / t(ms):  48.5 293.2 337.1)
e074-i0058 => L=9.268 acc= 90% / t(ms):  48.5 291.6 335.8)
e074-i0060 => L=9.366 acc= 92% / t(ms):  48.4 293.3 335.3)
e074-i0062 => L=9.140 acc= 95% / t(ms):  48.0 292.9 332.5)
e074-i0064 => L=8.667 acc= 96% / t(ms):  48.5 293.1 341.7)
e074-i0066 => L=8.757 acc= 89% / t(ms):  48.2 295.0 337.4)
e074-i0068 => L=8.971 acc= 87% / t(ms):  47.8 292.4 333.3)
e074-i0070 => L=9.494 acc= 95% / t(ms):  46.3 282.5 326.3)
e074-i0072 => L=8.901 acc= 95% / t(ms):  45.2 276.0 313.4)
e074-i0074 => L=8.969 acc= 92% / t(ms):  44.3 265.5 306.3)
e074-i0076 => L=9.435 acc= 92% / t(ms):  43.4 259.1 299.8)
e074-i0078 => L=8.944 acc= 86% / t(ms):  42.7 252.9 292.

e075-i0076 => L=9.485 acc= 91% / t(ms):  46.7 276.2 334.8)
e075-i0078 => L=9.018 acc= 92% / t(ms):  46.7 281.6 334.1)
e075-i0080 => L=9.112 acc= 86% / t(ms):  50.4 281.0 332.3)
e075-i0082 => L=8.547 acc= 98% / t(ms):  49.6 280.9 331.2)
e075-i0084 => L=8.560 acc= 96% / t(ms):  49.6 282.7 336.9)
e075-i0086 => L=8.690 acc= 97% / t(ms):  49.7 283.6 343.2)
e075-i0088 => L=8.729 acc= 94% / t(ms):  49.7 283.6 341.3)
e075-i0090 => L=8.705 acc= 91% / t(ms):  49.2 284.9 341.5)
e075-i0092 => L=9.716 acc= 91% / t(ms):  49.0 283.5 333.4)
e075-i0094 => L=9.080 acc= 92% / t(ms):  49.2 286.2 340.1)
e075-i0096 => L=9.191 acc= 93% / t(ms):  48.5 284.6 343.8)
e075-i0098 => L=9.082 acc= 93% / t(ms):  49.1 284.8 346.1)
e075-i0100 => L=9.078 acc= 96% / t(ms):  49.5 283.3 344.9)
e075-i0102 => L=8.727 acc= 95% / t(ms):  49.0 283.8 350.5)
e075-i0104 => L=9.183 acc= 92% / t(ms):  47.3 277.2 336.1)
e075-i0106 => L=8.875 acc= 94% / t(ms):  45.8 269.4 326.6)
e075-i0108 => L=9.168 acc= 94% / t(ms):  44.0 262.4 319.

e076-i0102 => L=10.018 acc= 89% / t(ms):  46.0 275.3 323.1)
e076-i0104 => L=9.660 acc= 91% / t(ms):  44.2 265.4 318.8)
e076-i0106 => L=9.112 acc= 94% / t(ms):  42.6 255.7 319.1)
e076-i0108 => L=8.725 acc= 95% / t(ms):  54.0 248.7 303.8)
e076-i0110 => L=9.265 acc= 93% / t(ms):  50.9 245.3 298.6)
e076-i0112 => L=8.903 acc= 89% / t(ms):  48.4 243.6 291.6)
e076-i0114 => L=8.989 acc= 92% / t(ms):  47.0 241.7 291.4)
e076-i0116 => L=8.924 acc= 97% / t(ms):  46.4 237.6 286.5)
e076-i0118 => L=8.788 acc= 88% / t(ms):  44.8 234.8 282.7)
e076-i0120 => L=8.480 acc= 96% / t(ms):  43.4 231.6 285.1)
e076-i0122 => L=9.012 acc= 94% / t(ms):  44.1 245.2 301.2)
e076-i0124 => L=9.442 acc= 93% / t(ms):  45.0 252.0 307.9)
e076-i0126 => L=8.883 acc= 95% / t(ms):  47.9 265.2 314.7)
e076-i0128 => L=9.248 acc= 95% / t(ms):  48.2 271.4 326.0)
e076-i0130 => L=9.255 acc= 92% / t(ms):  48.7 272.8 327.9)
e076-i0132 => L=9.034 acc= 93% / t(ms):  51.3 275.8 328.2)
e076-i0134 => L=8.566 acc= 97% / t(ms):  50.2 276.0 325